## Importar librerias

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Leer base

In [2]:
base_modelo = "../input/base_modelo.parquet"

In [3]:
df = pd.read_parquet(base_modelo)

In [4]:
df.head()

,periodo,id_cliente,tasa,monto_1m,monto_2m,monto_3m,monto_4m,monto_5m,monto_6m,cantidad_1m,...,segmento_1,segmento_2,segmento_3,segmento_4,segmento_5,segmento_6,segmento_7,segmento_8,segmento_9,segmento_10
16,202402,2,0.0344,7.817021,-16.118096,8.383090,8.793536,9.185740,7.697121,3.806662,...,0,0,1,0,0,0,0,0,0,0
17,202403,2,0.0344,5.150397,7.817021,-16.118096,8.383090,8.793536,9.185740,1.098612,...,0,0,1,0,0,0,0,0,0,0
18,202404,2,0.0344,9.188453,5.150397,7.817021,-16.118096,8.383090,8.793536,5.236442,...,0,0,1,0,0,0,0,0,0,0
19,202405,2,0.0344,9.099744,9.188453,5.150397,7.817021,-16.118096,8.383090,5.181784,...,0,0,1,0,0,0,0,0,0,0
35,202401,4,0.0344,7.431596,8.457210,8.055475,7.840706,7.498039,8.083637,4.110874,...,0,0,1,0,0,0,0,0,0,0


In [5]:
df.columns

Index(['periodo', 'id_cliente', 'tasa', 'monto_1m', 'monto_2m', 'monto_3m',
       'monto_4m', 'monto_5m', 'monto_6m', 'cantidad_1m', 'cantidad_2m',
       'cantidad_3m', 'cantidad_6m', 'frecuencia_1m', 'frecuencia_2m',
       'frecuencia_3m', 'ultima_compra_1m', 'ultima_compra_2m',
       'ultima_compra_3m', 'flg_churn', 'periodo_creacion', 'departamento',
       'segmento_pago', 'canal', 'crossell', 'tiempo_permanencia', 'flg_vip',
       'incidencias_a', 'incidencias_b', 'tipo_producto_producto_a',
       'tipo_producto_producto_b', 'tipo_producto_producto_c', 'segmento_0',
       'segmento_1', 'segmento_2', 'segmento_3', 'segmento_4', 'segmento_5',
       'segmento_6', 'segmento_7', 'segmento_8', 'segmento_9', 'segmento_10'],
      dtype='object')

## Entrenar hiperparametros - Linea base

In [6]:
# Separar las características y el objetivo
X = df[['periodo_creacion', 'flg_vip', 'tipo_producto_producto_a', 'tipo_producto_producto_b', 'tipo_producto_producto_c', 'monto_1m', 'monto_2m', 'monto_3m', 
        'cantidad_1m', 'cantidad_2m', 'cantidad_3m',
        'ultima_compra_1m', 'crossell']]  # Características
y = df['flg_churn']  # Variable objetivo

# Realizar el train-test split estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,  # 30% para el conjunto de prueba
    stratify=y,  # Estratificación por la variable objetivo
    random_state=42  # Semilla para reproducibilidad
)

## Entrenar modelos - Linea base

In [22]:
def entrenar_xgboost_rapido(X_train, X_test, y_train, y_test):
    """
    Entrena un modelo XGBoost para un problema de clasificación binaria utilizando
    parámetros óptimos previamente obtenidos.
    """
    # Parámetros óptimos (reemplaza con los que obtuviste en el GridSearch)
    parametros_optimos = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'use_label_encoder': False,
        'n_estimators': 100,
        'max_depth': 5,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 1,
        'random_state': 42
    }
    
    # Crear y entrenar el modelo
    xgb_model = xgb.XGBClassifier(**parametros_optimos)
    xgb_model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = xgb_model.predict(X_test)
    y_pred_prob = xgb_model.predict_proba(X_test)[:, 1]
    
    # Evaluar el modelo
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))
    print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_prob):.4f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    
    return xgb_model

In [23]:
def entrenar_random_forest_rapido(X_train, X_test, y_train, y_test):
    """
    Entrena un modelo Random Forest para un problema de clasificación binaria utilizando
    parámetros óptimos previamente obtenidos.
    """
    # Parámetros óptimos (reemplaza con los que obtuviste en el GridSearch)
    parametros_optimos = {
        'n_estimators': 100,
        'max_depth': 10,
        'min_samples_split': 5,
        'min_samples_leaf': 2,
        'max_features': 'sqrt',
        'random_state': 42,
        'n_jobs': -1
    }
    
    # Crear y entrenar el modelo
    rf_model = RandomForestClassifier(**parametros_optimos)
    rf_model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = rf_model.predict(X_test)
    y_pred_prob = rf_model.predict_proba(X_test)[:, 1]
    
    # Evaluar el modelo
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))
    print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_prob):.4f}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    
    return rf_model

In [24]:
modelo_final_randomforest = entrenar_random_forest_rapido(X_train, X_test, y_train, y_test)


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94    101158
           1       0.63      0.22      0.33     13696

    accuracy                           0.89    114854
   macro avg       0.77      0.60      0.64    114854
weighted avg       0.87      0.89      0.87    114854

AUC-ROC: 0.8875
Accuracy: 0.8918


In [25]:
modelo_final = entrenar_xgboost_rapido(X_train, X_test, y_train, y_test)


Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.91      0.98      0.94    101158
           1       0.60      0.28      0.38     13696

    accuracy                           0.89    114854
   macro avg       0.76      0.63      0.66    114854
weighted avg       0.87      0.89      0.87    114854

AUC-ROC: 0.8897
Accuracy: 0.8922
